In [1]:
from sklearn.ensemble import IsolationForest
from sklearn import svm
import pandas as pd
import numpy as np
import sys
import csv

params = {}
params['n_estimators'] = 100
params['max_samples'] ='auto'
params['contamination'] = 0.1
params['max_features'] = 1.0

params['nu'] = 0.001
params['gamma']='auto'
params['kernel'] ='poly'

params['path'] = 'data/train/B01.csv'
params['opath'] ='dataclean/B01.csv'
try:
    with open(params['path'],'r') as f:
        reader = csv.reader(f)
        head_row=next(reader)
        data_attribute = []
    for item in head_row:
        data_attribute.append(item)
    print(data_attribute)
except Exception as e:
    print(e)

['DE_time', 'FE_time', 'BA_time', 'RPM']


In [2]:
tn = pd.read_csv(params['path']) 
tn.dropna(inplace=True)
train = np.array(tn)
train_x = np.array(train)
print(train_x)
# clf = IsolationForest(n_estimators=params['n_estimators'], 
#                       max_samples=params['max_samples'], 
#                       contamination=params['contamination'], 
#                       max_features=params['max_features'], 
#                       bootstrap=False, n_jobs=1, random_state=None, 
#                       verbose=0).fit(train_x)
clf = svm.OneClassSVM(nu=params['nu'],
              kernel=params['kernel'],
              gamma=params['gamma']).fit(train_x)
pred = clf.predict(train_x)
print (pred.size)

[[ 1.91511018e-01 -3.69818182e-02 -8.20842730e-03  1.79600000e+03]
 [ 1.50577365e-01  4.49945455e-02 -1.55316320e-02  1.79600000e+03]
 [-3.68727745e-02 -8.62909091e-03  3.82255193e-03  1.79600000e+03]
 ...
 [ 2.42353214e-01 -6.98545455e-03  7.06970920e-02  1.79600000e+03]
 [ 3.24870259e-03  1.06836364e-02  7.12604154e-02  1.79600000e+03]
 [-6.22126547e-02 -1.01700000e-01 -2.03198813e-02  1.79600000e+03]]
14708


In [7]:
df = pd.DataFrame(pd.read_csv(params['path']))[0:pred.size]
df['pred']=pred
df2 = df[-df.pred.isin([-1])]
df2 = df2.drop(['pred'],axis=1)
data_out = df2.iloc[:,:].values
print(int(df2.size/4))
csvfile2 = open(params['opath'],'w',newline='')
writer = csv.writer(csvfile2)
writer.writerow(data_attribute)   #存属性
m=len(data_out)
for i in range(m):
    writer.writerow(data_out[i])

14708


In [66]:
import pandas as pd
import numpy as np
import os
from scipy import stats,signal,fftpack
import math
from pywt import wavedec
import traceback
import sys
columns_list=['time_mean','time_std','time_max','time_min','time_rms','time_ptp','time_median','time_iqr','time_pr','time_skew','time_kurtosis','time_var','time_amp',                'time_smr','time_wavefactor','time_peakfactor','time_pulse','time_margin','freq_mean','freq_std','freq_max','freq_min','freq_rms','freq_median',               'freq_iqr','freq_pr','freq_f2','freq_f3','freq_f4','freq_f5','freq_f6','freq_f7','freq_f8','ener_cA5','ener_cD1','ener_cD2','ener_cD3','ener_cD4',        'ener_cD5','ratio_cA5','ratio_cD1','ratio_cD2','ratio_cD3','ratio_cD4','ratio_cD5']
columns_list1 = [a + '_DE' for a in columns_list]
columns_list2 = [a + '_FE' for a in columns_list]
# columns_list3 = [a + '_BA' for a in columns_list]
# columns_list_final = columns_list1 + columns_list2+ columns_list3
columns_list_final = columns_list1 + columns_list2
params = {}
params['len_piece']=100 #窗口长度
windowlen=params['len_piece']


def feature_get(filepath,windowlen):
        df_data = pd.DataFrame(pd.read_csv(filepath))
#         dfs = df_data.loc[:,['DE_time','FE_time','BA_time']]
        dfs = df_data.loc[:,['DE_time','FE_time']]
        features_list=[]
        print(len(dfs))
        for i in range (0,len(dfs),windowlen):
            if(int((len(dfs)-i)/100)>=1): #舍去少量数据
                df=dfs[i:i+params['len_piece']]
                feature_list = []
    #             print(df)
                for i in df.columns:
                    #----------  time-domain feature,18
                    #依次为均值，标准差，最大值，最小值，均方根，峰峰值，中位数，四分位差，百分位差，偏度，峰度，方差，整流平均值，方根幅值，波形因子，峰值因子，脉冲值，裕度
                    df_line = df[i]
    #                 print(df_line)
                    time_mean = df_line.mean()
                    time_std = df_line.std()
                    time_max = df_line.max()
                    time_min = df_line.min()
                    time_rms = np.sqrt(np.square(df_line).mean())
                    time_ptp = time_max-time_min 
                    time_median = np.median(df_line)
                    time_iqr = np.percentile(df_line,75)-np.percentile(df_line,25)
                    time_pr = np.percentile(df_line,90)-np.percentile(df_line,10)
                    time_skew = stats.skew(df_line)
                    time_kurtosis = stats.kurtosis(df_line)
                    time_var = np.var(df_line)
                    time_amp = np.abs(df_line).mean()
                    time_smr = np.square(np.sqrt(np.abs(df_line)).mean())
                    #下面四个特征需要注意分母为0或接近0问题，可能会发生报错
                    time_wavefactor = time_rms/time_amp
                    time_peakfactor = time_max/time_rms
                    time_pulse = time_max/time_amp
                    time_margin = time_max/time_smr
                    #----------  freq-domain feature,15
                    #采样频率25600Hz
                    df_fftline = fftpack.fft(df[i])
                    freq_fftline = fftpack.fftfreq(len(df[i]),1/25600)
                    df_fftline = abs(df_fftline[freq_fftline>=0])
                    freq_fftline = freq_fftline[freq_fftline>=0]
                    #基本特征,依次为均值，标准差，最大值，最小值，均方根，中位数，四分位差，百分位差
                    freq_mean = df_fftline.mean()
                    freq_std = df_fftline.std()
                    freq_max = df_fftline.max()
                    freq_min = df_fftline.min()
                    freq_rms = np.sqrt(np.square(df_fftline).mean())
                    freq_median = np.median(df_fftline)
                    freq_iqr = np.percentile(df_fftline,75)-np.percentile(df_fftline,25)
                    freq_pr = np.percentile(df_fftline,90)-np.percentile(df_fftline,10)
                    #f2 f3 f4反映频谱集中程度
                    freq_f2 = np.square((df_fftline-freq_mean)).sum()/(len(df_fftline)-1)
                    freq_f3 = pow((df_fftline-freq_mean),3).sum()/(len(df_fftline)*pow(freq_f2,1.5))
                    freq_f4 = pow((df_fftline-freq_mean),4).sum()/(len(df_fftline)*pow(freq_f2,2))
                    #f5 f6 f7 f8反映主频带位置
                    freq_f5 = np.multiply(freq_fftline,df_fftline).sum()/df_fftline.sum()
                    freq_f6 = np.sqrt(np.multiply(np.square(freq_fftline),df_fftline).sum())/df_fftline.sum()
                    freq_f7 = np.sqrt(np.multiply(pow(freq_fftline,4),df_fftline).sum())/np.multiply(np.square(freq_fftline),df_fftline).sum()
                    freq_f8 = np.multiply(np.square(freq_fftline),df_fftline).sum()/np.sqrt(np.multiply(pow(freq_fftline,4),df_fftline).sum()*df_fftline.sum())
                    #----------  timefreq-domain feature,12
                    #5级小波变换，最后输出6个能量特征和其归一化能量特征
                    cA5, cD5, cD4, cD3, cD2, cD1 = wavedec(df[i], 'db10', level=5)
                    ener_cA5 = np.square(cA5).sum()
                    ener_cD5 = np.square(cD5).sum()
                    ener_cD4 = np.square(cD4).sum()
                    ener_cD3 = np.square(cD3).sum()
                    ener_cD2 = np.square(cD2).sum()
                    ener_cD1 = np.square(cD1).sum()
                    ener = ener_cA5 + ener_cD1 + ener_cD2 + ener_cD3 + ener_cD4 + ener_cD5
                    ratio_cA5 = ener_cA5/ener
                    ratio_cD5 = ener_cD5/ener
                    ratio_cD4 = ener_cD4/ener
                    ratio_cD3 = ener_cD3/ener
                    ratio_cD2 = ener_cD2/ener
                    ratio_cD1 = ener_cD1/ener
                    feature_list.extend([time_mean,time_std,time_max,time_min,time_rms,time_ptp,time_median,time_iqr,time_pr,time_skew,time_kurtosis,time_var,time_amp,
                                     time_smr,time_wavefactor,time_peakfactor,time_pulse,time_margin,freq_mean,freq_std,freq_max,freq_min,freq_rms,freq_median,
                                     freq_iqr,freq_pr,freq_f2,freq_f3,freq_f4,freq_f5,freq_f6,freq_f7,freq_f8,ener_cA5,ener_cD1,ener_cD2,ener_cD3,ener_cD4,ener_cD5,
                                     ratio_cA5,ratio_cD1,ratio_cD2,ratio_cD3,ratio_cD4,ratio_cD5])
                features_list.append(feature_list)
        return features_list

for i in range(10,15):
    params['data_path'] = 'data/train/OR'+str(i)+'.csv'
    params['opath'] ='datafeature/OR'+str(i)+'_feature.csv'
#     params['data_path'] = 'data/train/NORMAL0'+str(i)+'.csv'
#     params['opath'] ='datafeature/NORMAL0'+str(i)+'_feature.csv'
    fea = feature_get(params['data_path'],windowlen)
    # print(fea)
    result = pd.DataFrame(fea,columns = columns_list_final)
    result['label']=3
    #B:1;IR:2;OR:3;Normal:0.
    result.to_csv(params['opath'],index=False,header=True)
print("done")

7319
7328
7319
7319
7345
done


In [24]:
import pandas as pd
path = 'datafeature/B01_feature.csv'  #特征提取后的csv文件路径
df = pd.DataFrame(pd.read_csv(path))
delete_features = [] #需要删除的列名自行加到数组里
df = df.drop(delete_features, axis=1) #特征选择之后的数据

In [57]:
import pandas as pd
df1 = pd.read_csv('datafeature/B01_feature.csv')
df2 = pd.read_csv('datafeature/B02_feature.csv')
df3 = pd.read_csv('datafeature/B03_feature.csv')
df4 = pd.read_csv('datafeature/B04_feature.csv')
df5 = pd.read_csv('datafeature/B05_feature.csv')
df_fault = pd.concat([df1,df2,df3,df4,df5])
df_fault = df_fault.reset_index(drop=True)
df_fault.to_csv('data_faultB.csv',index=False)

In [73]:
import os
import pandas as pd
filePath='datafeature'
outputfile='train.csv'
fileList = os.listdir(filePath)
# print(fileList)
# print(len(fileList))
path = os.path.join(filePath, fileList[0])
df = pd.read_csv(path)
df.to_csv(outputfile,index=False, header=True)
for i in range (1,len(fileList)):
    path = os.path.join(filePath, fileList[i])
#     print(path)
    df = pd.read_csv(path)
    df.to_csv(outputfile, mode='a', index=False, header=False)

['B01_feature.csv', 'B02_feature.csv', 'B03_feature.csv', 'B04_feature.csv', 'B05_feature.csv', 'B06_feature.csv', 'IR01_feature.csv', 'IR02_feature.csv', 'IR03_feature.csv', 'IR04_feature.csv', 'IR05_feature.csv', 'IR06_feature.csv', 'NORMAL01_feature.csv', 'NORMAL02_feature.csv', 'OR01_feature.csv', 'OR02_feature.csv', 'OR03_feature.csv', 'OR04_feature.csv', 'OR05_feature.csv', 'OR06_feature.csv', 'OR07_feature.csv', 'OR08_feature.csv', 'OR09_feature.csv', 'OR10_feature.csv', 'OR11_feature.csv', 'OR12_feature.csv', 'OR13_feature.csv', 'OR14_feature.csv']
28


In [76]:
import lightgbm as lgb
import joblib
def train():
    # 模型初始化，设置random_state保证可复现性，便于观察优化
    train_data = pd.read_csv('train.csv')
    train_data_y = train_data['label']
    # 除去标签的所有列就是特征
    train_data_x = train_data.drop(['label'], axis=1)
    model_lgb_default = lgb.LGBMClassifier(random_state=2019)
    # 模型训练
    model_lgb_default.fit(train_data_x, train_data_y)
    joblib.dump(model_lgb_default, 'model/lightgbm_model.model')

train()

In [ ]:
def test_lightgbm():
    # 加载模型
    model = joblib.load('model/lightgbm_model.model')
    